In [10]:
import numpy as np
import matplotlib.pyplot as plt
import os

### **Extract Species from *species.inp*** 

In [11]:
species_inp = []
with open('../STAGRAD-main/input/species.inp') as f:
    for line in f.readlines():
        species_inp.append(line)
        
nsp = int(species_inp[10].split()[0])
ntv = int(species_inp[10].split()[1])
ntr = int(species_inp[10].split()[2])

species_name_list = []
for i in range(nsp):
    species_name_list.append(species_inp[15 + i].split()[0])

print(species_name_list)

['N', 'O', 'Np', 'Op', 'N2', 'O2', 'NO', 'N2p', 'O2p', 'NOp', 'e']


### **Define Indices for All Quantities**

In [12]:
def unpack_output():
    data_dict = {}

    data_flow = np.genfromtxt('../STAGRAD-main/output_flow/outflow00.plt')
    ind_px, ind_rho, ind_pres, ind_ttr, ind_velr, ind_velt, ind_amach = [i for i in range(np.shape(data_flow)[1])]

    data_engy = np.genfromtxt('../STAGRAD-main/output_flow/outengy00.plt')
    ind_px, ind_ttr, ind_tvib, ind_trot, ind_etot, ind_evib, ind_erot, ind_htot = [i for i in range(8)]
    ind_hsp = [ind_htot + i + 1 for i in range(nsp)]
    ind_pr, ind_hs, ind_hsw, ind_ht, ind_hr, ind_hsht_sum = [max(ind_hsp) + i + 1 for i in range(6)]

    data_spes = np.genfromtxt('../STAGRAD-main/output_flow/outspes00.plt')
    ind_px, ind_rho = 0, 1
    ind_rhosp = [ind_rho + i + 1 for i in range(nsp)]
    ind_frac = [max(ind_rhosp) + i + 1 for i in range(nsp)]
    ind_amfrac = [max(ind_frac) + i + 1 for i in range(nsp)]

    data_tran = np.genfromtxt('../STAGRAD-main/output_flow/outtran00.plt')
    ind_px, ind_vis_c, ind_etrk_c, ind_eevk_c, ind_erk_c = [i for i in range(5)]
    ind_dif_c = [ind_erk_c + i + 1 for i in range(nsp)]

    
    data_dict['Distance from Wall'] = data_flow[:, ind_px]
    data_dict['Total Density'] = data_flow[:, ind_rho]
    data_dict['Pressure'] = data_flow[:, ind_pres]
    data_dict['Radial Velocity'] = data_flow[:, ind_velr]
    data_dict['Theta Velocity'] = data_flow[:, ind_velt]
    data_dict['Mach Number'] = data_flow[:, ind_amach]
    data_dict['Translational Temperature'] = data_engy[:, ind_ttr]
    data_dict['Vibrational Temperature'] = data_engy[:, ind_tvib]
    data_dict['Rotational Temperature'] = data_engy[:, ind_trot]
    data_dict['Total Energy'] = data_engy[:, ind_etot]
    data_dict['Vibrational Energy'] = data_engy[:, ind_evib]
    data_dict['Rotational Energy'] = data_engy[:, ind_erot]
    data_dict['Total Enthalpy'] = data_engy[:, ind_htot]
    data_dict['prandtl number'] = data_engy[:, ind_pr]
    data_dict['averaged static enthalpy'] = data_engy[:, ind_hs]
    data_dict['averaged static enthalpy calculated by wall tempertaure T_w'] = data_engy[:, ind_hsw] 
    data_dict['averaged total enthalpy'] = data_engy[:, ind_ht] 
    data_dict['recovery enthalpy'] = data_engy[:, ind_hr] 
    data_dict['hsht sum'] = data_engy[: ind_hsht_sum]
    data_dict['mixture viscosity'] = data_tran[: , ind_vis_c]
    data_dict['mixture thermal conductivity for eq mode'] = data_tran[: , ind_etrk_c]
    data_dict['mixture thermal conductivity for electron-electronic-vibrational mode'] = data_tran[: , ind_eevk_c]
    data_dict['mixture thermal conductivity for rotaional mode'] = data_tran[: , ind_erk_c]






    
    for i, species in enumerate(species_name_list):
        data_dict[f'{species} Enthalpy'] = data_engy[:, ind_hsp[i]]
    for i, species in enumerate(species_name_list):
        data_dict[f'{species} Density'] = data_spes[:, ind_rhosp[i]]
    for i, species in enumerate(species_name_list):
        data_dict[f'{species} Fraction'] = data_spes[:, ind_frac[i]] 
    for i, species in enumerate(species_name_list):
        data_dict[f'{species} Mass fraction'] = data_spes[:, ind_amfrac[i]]
    for i, species in enumerate(species_name_list):
        data_dict[f'{species} Diffusion'] = data_tran[:, ind_dif_c[i]]

    return data_dict

### **Plotter Setup**

In [13]:
%matplotlib qt
data_dict = unpack_output()

#plot types (as string): temperature, fraction
def plotter(plot_type):
    '''
    Create plots for temperatures or species fraction
    Valid plot types (as string): 'temperature', 'fraction'
    '''
    fig, ax = plt.subplots(figsize = (10, 5))

    if plot_type == 'temperature':
        temperature_list = ['Translational Temperature', 'Rotational Temperature', 'Vibrational Temperature']
        for i, temperature in enumerate(temperature_list):
            ax.plot(data_dict['Distance from Wall'], data_dict[temperature], label = f'{temperature}')
        ax.set_xlim(-0.2, 0)
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('Temperature (K)')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
    elif plot_type == 'enthalpy':
        enthalpy_list = ['averaged static enthalpy', 'averaged total enthalpy', 'recovery enthalpy']
        for i, enthalpy in enumerate(enthalpy_list):
            ax.plot(data_dict['Distance from Wall'], data_dict[enthalpy], label = f'{enthalpy}')
        ax.set_xlim(-0.04, 0)
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('enthalpy (J/Kg)')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
    elif plot_type == 'thermal conductivity':
        conductivity_list = ['mixture thermal conductivity for eq mode','mixture thermal conductivity for electron-electronic-vibrational mode' , 'mixture thermal conductivity for rotaional mode']
        for i, conductivity in enumerate(conductivity_list):
            ax.plot(data_dict['Distance from Wall'], data_dict[conductivity], label = f'{conductivity}')
        ax.set_xlim(-0.04, 0)
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('Thermal conductivity (mW/cm-K)')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
    elif plot_type == 'mole fraction':
        for i, species in enumerate(species_name_list):
            ax.plot(data_dict['Distance from Wall'], data_dict[f'{species} ' + 'Fraction'], label = f'{species} ' + 'Fraction')
        ax.set_xlim(-0.1, 0)
        #ax.set_yscale('log')
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('Species mole Fraction')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
    elif plot_type == 'Total Density':
        ax.plot(data_dict['Distance from Wall'], data_dict['Total Density'], label = 'Total Density')
        ax.set_xlim(-0.04, 0)
        #ax.set_yscale('log')
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('Total Density (Kg/m3)')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
    elif plot_type == 'static Pressure':
        ax.plot(data_dict['Distance from Wall'], data_dict['Pressure'], label = 'static Pressure')
        ax.set_xlim(-0.04, 0)
        #ax.set_yscale('log')
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('static Pressure (Pa)')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
    elif plot_type == 'Mach Number':
        ax.plot(data_dict['Distance from Wall'], data_dict['Mach Number'], label = 'Mach Number')
        ax.set_xlim(-0.04, 0)
        #ax.set_yscale('log')
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('Mach Number')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
    elif plot_type == 'Mass fraction':
        for i, species in enumerate(species_name_list):
            ax.plot(data_dict['Distance from Wall'], data_dict[f'{species} ' + 'Mass fraction'], label = f'{species} ' + 'Mass fraction')
        ax.set_xlim(-0.2, 0)
        #ax.set_yscale('log')
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('Species  Mass Fraction')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
    elif plot_type == 'Effective diffusion coefficient':
        for i, species in enumerate(species_name_list):
            ax.plot(data_dict['Distance from Wall'], data_dict[f'{species} ' + 'Diffusion'], label = f'{species} ' + 'Diffusion')
        ax.set_xlim(-0.1, 0)
        #ax.set_yscale('log')
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('Species effective diffusion coefficient (cm2/s)')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
    elif plot_type == 'mixture viscosity':
        ax.plot(data_dict['Distance from Wall'], data_dict['mixture viscosity'], label = 'mixture viscosity')
        ax.set_xlim(-0.04, 0)
        #ax.set_yscale('log')
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('Mixture viscosity (mPoise)')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
    elif plot_type == 'Species densities':
        for i, species in enumerate(species_name_list):
            ax.plot(data_dict['Distance from Wall'], data_dict[f'{species} ' + 'Density'], label = f'{species} ' + 'Density')
        ax.set_xlim(-0.1, 0)
        #ax.set_yscale('log')
        #ax.set_ylim(1e-5)
        ax.set_xlabel('Normal Distance from Stagnation Point (m)')
        ax.set_ylabel('Species densities (Kg/m3)')
        ax.legend(loc='upper left', bbox_to_anchor=(1.025, 1),
                    ncol=2, borderaxespad=0, fancybox=True, shadow=True)
        fig.subplots_adjust(right=0.55)
        
    ax.autoscale(axis='y', tight=True)
    leg = interactive_legend()
    return fig, ax, leg

class InteractiveLegend(object):
    '''
    Set up interactive legend
    '''
    def __init__(self, legend):
        self.legend = legend
        self.fig = legend.axes.figure

        self.lookup_artist, self.lookup_handle = self._build_lookups(legend)
        self._setup_connections()

        self.update()

    def _setup_connections(self):
        for artist in self.legend.texts + self.legend.legendHandles:
            artist.set_picker(10) # 10 points tolerance

        self.fig.canvas.mpl_connect('pick_event', self.on_pick)
        self.fig.canvas.mpl_connect('button_press_event', self.on_click)

    def _build_lookups(self, legend):
        labels = [t.get_text() for t in legend.texts]
        handles = legend.legendHandles
        label2handle = dict(zip(labels, handles))
        handle2text = dict(zip(handles, legend.texts))

        lookup_artist = {}
        lookup_handle = {}
        for artist in legend.axes.get_children():
            if artist.get_label() in labels:
                handle = label2handle[artist.get_label()]
                lookup_handle[artist] = handle
                lookup_artist[handle] = artist
                lookup_artist[handle2text[handle]] = artist

        lookup_handle.update(zip(handles, handles))
        lookup_handle.update(zip(legend.texts, handles))

        return lookup_artist, lookup_handle

    def on_pick(self, event):
        handle = event.artist
        if handle in self.lookup_artist:

            artist = self.lookup_artist[handle]
            artist.set_visible(not artist.get_visible())
            self.update()

    def on_click(self, event):
        if event.button == 3:
            visible = False
        elif event.button == 2:
            visible = True
        else:
            return

        for artist in self.lookup_artist.values():
            artist.set_visible(visible)
        self.update()

    def update(self):
        for artist in self.lookup_artist.values():
            handle = self.lookup_handle[artist]
            if artist.get_visible():
                handle.set_visible(True)
            else:
                handle.set_visible(False)
        self.fig.canvas.draw()

    def show(self):
        plt.show()

def interactive_legend(ax=None):
    if ax is None:
        ax = plt.gca()
    if ax.legend_ is None:
        ax.legend()

    return InteractiveLegend(ax.get_legend())

### **Run Plots**

In [14]:
#plot types: 'temperature', 'mole fraction', 'Total Density', 'Mach Number', 'Mass fraction', 'static Pressure', 'enthalpy',
#plot types: 'thermal conductivity', 'mixture viscosity' , 'Effective diffusion coefficient' , 'Species densities'
if __name__ == '__main__':
    fig, ax, leg = plotter('Mass fraction')
    plt.show()

In [15]:
# Define the file path
file_path = '../STAGRAD-main/output_flow/qwflux00.plt'  # Replace this with the path to your actual file

# Check if the file exists
if os.path.exists(file_path):
    # Open and read the file
    with open(file_path, 'r') as file:
        # Read all lines in the file
        lines = file.readlines()
        # Extract the last line
        last_line = lines[-1] if lines else None

    # Check if there was a last line to read
    if last_line:
        # Split the last line into columns based on the delimiter
        columns = last_line.strip().split('  ')  # Adjust the delimiter if needed

        # Descriptions for each column based on your specifications
        descriptions = [
            "Iteration number",
            "Equilibrium surface heat flux in W/m2",
            "Nonequilibrium surface heat flux in W/m2",
            "Difference between nonequilibrium surface heat flux and mass diffusion heat flux in W/m2",
            "Mass diffusion heat flux in W/m2",
            "Translational heat flux in W/m2",
            "El-ex-vib heat flux in W/m2",
            "Rotational heat flux in W/m2"
        ]

        # Print each column with its description
        for i, description in enumerate(descriptions):
            if i < len(columns):
                print(f"{description}: '{columns[i]}'")
            else:
                print(f"{description}: 'No data available'")
    else:
        print("The file is empty or has no readable last line.")
else:
    print("File does not exist. Please check the file path.")



Iteration number: '39300'
Equilibrium surface heat flux in W/m2: ' 1.5336E+05'
Nonequilibrium surface heat flux in W/m2: ' 1.6977E+05'
Difference between nonequilibrium surface heat flux and mass diffusion heat flux in W/m2: ' 1.6971E+05'
Mass diffusion heat flux in W/m2: ' 5.7810E+01'
Translational heat flux in W/m2: ' 1.5021E+05'
El-ex-vib heat flux in W/m2: ' 1.9499E+04'
Rotational heat flux in W/m2: ' 0.0000E+00'


In [16]:
# Define the file path
file_path = '../STAGRAD-main/output_flow/wall_prop.plt'  # Replace this with the path to your actual file

# Check if the file exists
if os.path.exists(file_path):
    # Open and read the file
    with open(file_path, 'r') as file:
        # Read all lines in the file
        lines = file.readlines()
        # Extract the last line
        last_line = lines[-1] if lines else None

    # Check if there was a last line to read
    if last_line:
        # Split the last line into columns based on the delimiter
        columns = last_line.strip().split('  ')  # Adjust the delimiter if needed

        # Basic column descriptions
        descriptions = [
            "Iteration number",
            "Wall static enthalpy",
            "Wall total enthalpy"
        ]

        # Extend descriptions with species-specific enthalpies
        for i in range(1, nsp + 1):
            descriptions.append(f"{species_name_list[i-1]} specific enthalpy")

        # Further descriptions for remaining properties
        additional_descriptions = [
            "Wall translational temperature",
            "Wall vibrational temperature",
            "Wall rotational temperature",
            "Wall total pressure",
            "Wall total density",
        ]

        # Append additional descriptions at the end
        descriptions.extend(additional_descriptions)

        # Extend descriptions with species-mass fractions
        for i in range(1, nsp + 1):
            descriptions.append(f"{species_name_list[i-1]} mass fraction")


        # Print each column with its description
        for i, description in enumerate(descriptions):
            if i < len(columns):
                print(f"{description}: '{columns[i]}'")
            else:
                print(f"{description}: 'No data available'")
    else:
        print("The file is empty or has no readable last line.")
else:
    print("File does not exist. Please check the file path.")




Iteration number: '39300'
Wall static enthalpy: ' 1.2507E+07'
Wall total enthalpy: ' 1.2507E+07'
N specific enthalpy: ' 3.4328E+07'
O specific enthalpy: ' 1.6088E+07'
Np specific enthalpy: ' 1.3489E+08'
Op specific enthalpy: ' 9.8562E+07'
N2 specific enthalpy: ' 4.1470E+05'
O2 specific enthalpy: ' 3.7899E+05'
NO specific enthalpy: ' 3.4196E+06'
N2p specific enthalpy: ' 5.4309E+07'
O2p specific enthalpy: ' 3.6971E+07'
NOp specific enthalpy: ' 3.3407E+07'
e specific enthalpy: ' 1.4848E+10'
Wall translational temperature: ' 6.9000E+02'
Wall vibrational temperature: ' 6.9000E+02'
Wall rotational temperature: ' 6.9000E+02'
Wall total pressure: ' 5.7617E+02'
Wall total density: ' 1.9754E-03'
N mass fraction: ' 2.4763E-01'
O mass fraction: ' 2.3465E-01'
Np mass fraction: ' 1.5039E-05'
Op mass fraction: ' 1.3707E-04'
N2 mass fraction: ' 5.1709E-01'
O2 mass fraction: ' 5.6873E-05'
NO mass fraction: ' 4.2091E-04'
N2p mass fraction: ' 8.8549E-11'
O2p mass fraction: ' 2.4945E-09'
NOp mass fraction

In [17]:
# Define the file path
#file_path = '../STAGRAD-main/output_flow/stand_off.dat'  # Replace this with the path to your actual file

# Check if the file exists
#if os.path.exists(file_path):
    # Open and read the file
#    with open(file_path, 'r') as file:
        # Read all lines in the file
#        lines = file.readlines()

#for i in range(len(lines)) : 
#    print(lines[i])

***

In [18]:
#flow: coord, rho, pres, temp, velr, velt, mach
#spes coord, rho, rhosp (nsp), frac (nsp), amfrac (nsp)
#engy: coord, ttr, tvib, trot, etot, evib, erot, (etot+pres)/rho??, hsp (species enthalpy, columns = nsp), pr, hs, hsw, ht, hr, hs + 0.69**0.3333(ht-hs)